In [149]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import pylab
import random
import statistics
import numpy as np

In [141]:
data = pd.read_csv("cloth/edges_ln.csv")
data.rename(columns={"balance" : "capacity", "fee_proportional" : "weight"}, inplace=True)
data = data[['from_node_id', 'to_node_id', 'capacity', 'weight']]

## preprocess
data = data.groupby(['from_node_id', 'to_node_id']).agg({'capacity': 'sum', 'weight': 'mean'}).reset_index()
data = data.astype({'weight' : int})
# data[(data['from_node_id'] == 1)]['capacity'].sum()
data

,from_node_id,to_node_id,capacity,weight
0,0,1,4300909,1
1,1,0,45699091,1
2,1,2,607791280,1
3,1,6,93361068,1
4,1,12,5393325728,1
...,...,...,...,...
54195,6002,345,19782854,1
54196,6002,854,187850564,1
54197,6003,5765,6738344,1
54198,6004,1931,235598450,1


In [142]:
G = nx.from_pandas_edgelist(data, 'from_node_id', 'to_node_id', edge_attr=['capacity', 'weight'], create_using=nx.DiGraph())

In [156]:
%%time
sampleSize = 300
randS = random.sample(range(0, 6005), sampleSize)
randT = random.sample(range(0, 6005), sampleSize)
X = []
costCoef = 1
for i in range(sampleSize):
    s, t = randS[i], randT[i]
    mincostFlow = nx.max_flow_min_cost(G, s, t)
    mincost = nx.cost_of_flow(G, mincostFlow)
    mincost /= 1000000
    mincostFlowValue = sum((mincostFlow[u][t] for u in G.predecessors(t))) - sum(
        (mincostFlow[t][v] for v in G.successors(t))
    )
    
    fund = sum((G[s][u]['capacity'] for u in G.predecessors(s)))
    
    X.append((mincostFlowValue - costCoef * mincost) / fund)

print(X)

[0.9988451739498952, 0.999988052008695, 0.999998, 0.07076843194389884, 0.9999979999999999, 0.9999982004327114, 0.9999970862377757, 0.9999972659811799, 0.999998, 0.999997, 0.031549865103154856, 0.9999970000000001, 0.9999882490699122, 0.0673334244889245, 1.5888562905073175e-05, 0.9999969999999999, 0.999997063704086, 0.8322973137128814, 0.03540916507077767, 0.01009434614707234, 0.999993, 0.9999968398365159, 0.999986, 0.831178311537423, 0.9982676931817922, 0.999997, 0.001334381947821882, 5.243772320551752e-05, 0.5321785867646567, 0.999999, 0.9999968265541157, 0.9999962249432733, 0.5210662425350687, 0.13226205828294849, 0.999997, 0.002202263355338986, 0.9999955828229705, 0.018614573484240744, 0.077546957181767, 0.9578589277492353, 0.37568159148479796, 0.9999969999999999, 0.0012497127426532788, 0.0011875604965991982, 0.5991376789434218, 0.9999960600052384, 0.006801351605316772, 0.8114248571818307, 0.8735621018788834, 2.6917472571352872e-05, 0.9999969999999999, 0.9999969999999999, 4.555200811

In [157]:
print("overall std :",np.std(X) / np.sqrt(sampleSize))
print("Lightning network EF :", np.mean(X))

overall std : 0.02562509519109166
Lightning network EF : 0.5508970283112261
